In [108]:
import pandas as pd
import json

# Read the JSON file line by line (json lines format)
with open('data/matches/all_matches_from_tournaments.json', 'r', encoding='utf-8') as f:
	lines = f.readlines()
	data = [json.loads(line) for line in lines]

df = pd.DataFrame(data)

In [109]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 58 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   version                  1553 non-null   int64  
 1   match_id                 1553 non-null   int64  
 2   draft_timings            1553 non-null   object 
 3   teamfights               1553 non-null   object 
 4   objectives               1553 non-null   object 
 5   chat                     1553 non-null   object 
 6   radiant_gold_adv         1553 non-null   object 
 7   radiant_xp_adv           1553 non-null   object 
 8   cosmetics                1553 non-null   object 
 9   players                  1553 non-null   object 
 10  leagueid                 1553 non-null   int64  
 11  start_time               1553 non-null   int64  
 12  duration                 1553 non-null   int64  
 13  series_id                1553 non-null   int64  
 14  series_type             

In [111]:
import subprocess

def run_get_glicko_for_dates(dates):
	for date in dates:
		# Convert date to string if it's a datetime.date object
		date_str = str(date)
		subprocess.run(['python', 'get_glicko.py', date_str])

run_get_glicko_for_dates(df['tournament_start_date'].unique())

In [112]:
df_players = df[['match_id', 'start_time', 'players', 'tournament_start_date', 'radiant_team_id', 'dire_team_id']].explode('players').reset_index(drop=True)
df_players = pd.concat(
	[df_players[['match_id', 'start_time', 'tournament_start_date', 'radiant_team_id', 'dire_team_id']], pd.DataFrame(df_players['players'].tolist())],
	axis=1
)

In [113]:
df_players['tournament_start_date']

0        2024-09-04
1        2024-09-04
2        2024-09-04
3        2024-09-04
4        2024-09-04
            ...    
15525    2025-07-08
15526    2025-07-08
15527    2025-07-08
15528    2025-07-08
15529    2025-07-08
Name: tournament_start_date, Length: 15530, dtype: object

In [114]:
df_players.rename(columns={'start_time': 'match_start_time'}, inplace=True)

In [115]:
df_players.head(1)

,match_id,match_start_time,tournament_start_date,radiant_team_id,dire_team_id,player_slot,obs_placed,sen_placed,creeps_stacked,camps_stacked,rune_pickups,firstblood_claimed,teamfight_participation,towers_killed,roshans_killed,observers_placed,stuns,max_hero_hit,times,gold_t,lh_t,dn_t,xp_t,obs_log,sen_log,obs_left_log,sen_left_log,purchase_log,kills_log,buyback_log,runes_log,connection_log,lane_pos,obs,sen,actions,pings,purchase,gold_reasons,xp_reasons,killed,item_uses,ability_uses,ability_targets,damage_targets,hero_hits,damage,damage_taken,damage_inflictor,runes,killed_by,kill_streaks,multi_kills,life_state,healing,damage_inflictor_received,randomed,pred_vict,neutral_tokens_log,party_id,permanent_buffs,party_size,account_id,team_number,team_slot,hero_id,hero_variant,item_0,item_1,item_2,item_3,item_4,item_5,backpack_0,backpack_1,...,deaths,assists,leaver_status,last_hits,denies,gold_per_min,xp_per_min,level,net_worth,aghanims_scepter,aghanims_shard,moonshard,hero_damage,tower_damage,hero_healing,gold,gold_spent,ability_upgrades_arr,personaname,name,last_login,rank_tier,is_subscriber,radiant_win,match_start_time,duration,cluster,lobby_type,game_mode,is_contributor,patch,region,isRadiant,win,lose,total_gold,total_xp,kills_per_min,kda,abandons,neutral_kills,tower_kills,courier_kills,lane_kills,hero_kills,observer_kills,sentry_kills,roshan_kills,necronomicon_kills,ancient_kills,buyback_count,observer_uses,sentry_uses,lane_efficiency,lane_efficiency_pct,lane,lane_role,is_roaming,purchase_time,first_purchase_time,item_win,item_usage,purchase_ward_observer,purchase_ward_sentry,actions_per_min,life_state_dead,cosmetics,benchmarks,purchase_tpscroll,purchase_gem,performance_others,purchase_rapier,additional_units,neutral_item_history,item_neutral2
0,7944311818,1726415990,2024-09-04,8599101,2163,0,7,12,6,2,3,0,0.647059,0,0,7,29.466703,"{'type': 'max_hero_hit', 'time': 1029, 'max': ...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...","[0, 231, 550, 640, 770, 1113, 1203, 1409, 1580...","[0, 2, 8, 9, 10, 10, 10, 10, 10, 10, 11, 12, 1...","[0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[6, 68, 575, 665, 839, 1085, 1149, 1227, 1846,...","[{'time': -34, 'type': 'obs_log', 'slot': 0, '...","[{'time': 33, 'type': 'sen_log', 'slot': 0, 'x...","[{'time': 326, 'type': 'obs_left_log', 'slot':...","[{'time': 453, 'type': 'sen_left_log', 'slot':...","[{'time': -89, 'key': 'smoke_of_deceit'}, {'ti...","[{'time': 260, 'key': 'npc_dota_hero_dark_will...","[{'time': 1358, 'slot': 0, 'type': 'buyback_lo...","[{'time': 211, 'key': 5}, {'time': 841, 'key':...",[],"{'76': {'76': 4}, '77': {'77': 1, '80': 1}, '7...","{'80': {'151': 1}, '98': {'157': 1}, '108': {'...","{'72': {'127': 1}, '79': {'150': 1}, '81': {'1...","{'1': 4405, '2': 25, '3': 6, '4': 425, '5': 27...",63.0,"{'ward_dispenser': 11, 'smoke_of_deceit': 3, '...","{'0': 659, '1': -670, '6': 350, '11': 90, '12'...","{'0': 135, '1': 3606, '2': 7415, '4': 1960}","{'npc_dota_courier': 1, 'npc_dota_creep_badguy...","{'smoke_of_deceit': 2, 'ward_dispenser': 21, '...","{'ability_lamp_use': 10, 'marci_grapple': 17, ...",{'marci_grapple': {'npc_dota_hero_dark_willow'...,{'marci_grapple': {'npc_dota_hero_dark_willow'...,"{'marci_grapple': 17, 'null': 90, 'marci_compa...","{'npc_dota_hero_dark_willow': 1260, 'npc_dota_...","{'npc_dota_hero_dark_willow': 3886, 'npc_dota_...","{'marci_grapple': 793, 'null': 7038, 'marci_co...","{'5': 1, '8': 2}","{'npc_dota_hero_visage': 2, 'npc_dota_hero_puc...",{},{},"{'0': 1890, '1': 12, '2': 167}","{'npc_dota_hero_marci': 10478, 'npc_dota_hero_...","{'null': 11767, 'dark_willow_bramble_maze': 82...",False,False,[],0,[],10,91730177,0,0,136,1,229,29,73,11,73,73,0,0,...,7,8,0,36,2,248,379,15,5746,0,0,0,10715,0,8182,1031,6835,"[8192, 8235, 1426, 8235, 8235, 8198, 8235, 142...",Seleri,Seleri,2020-10-05T18:16:42.773Z,80.0,False,False,1726415990,2068,187,1,2,False,56,8.0,True,0,1,8547,13062,0.087041,1.38,0,2,0,1,24,3,1,4,0,0,0,2,5,4,0.418957,41.0,1,1,False,"{'smoke_of_de

In [116]:
df_players_name_none_unique = df_players[df_players['name'].isna() & df_players['account_id'].notna()]
df_players_name_none_unique = df_players_name_none_unique['account_id'].unique()

In [117]:
df_players_name_none_unique.tolist()

[164962869,
 116585378,
 369342470,
 171097887,
 115464954,
 48823667,
 208181197,
 181267255,
 120613892,
 140251702,
 295697470,
 910028605,
 114619230,
 296702734,
 276449590,
 1048617659,
 94786276,
 1150772339,
 97658618,
 137193239,
 223342537,
 117514269,
 399804216,
 238239590,
 439345730,
 392006194,
 333351042,
 200573858,
 9403474]

164962869 Shad
116585378 MidOne
369342470 SoNNeikO
171097887 dEsire
115464954 Nande
48823667 Bengan
208181197 Adzantick
181267255 OneJey
120613892 W1sh-
140251702 Mirage`
295697470 Immersion
94786276 Nine
1150772339 mangekyou
97658618 Timado
137193239 Paparazi灬
223342537 lupsione
117514269 laise
399804216 Serenada
238239590 Bryle
439345730 reibl
392006194 ififall
333351042 mrls
9403474 yamich

In [118]:
# Mapping from account_id to player name
account_id_to_name = {
	164962869: "Shad",
	116585378: "MidOne",
	369342470: "SoNNeikO",
	171097887: "dEsire",
	115464954: "Nande",
	48823667: "Bengan",
	208181197: "Adzantick",
	181267255: "OneJey",
	120613892: "W1sh-",
	140251702: "Mirage",
	295697470: "Immersion",
	94786276: "Nine",
	1150772339: "mangekyou",
	97658618: "Timado",
	137193239: "Paparazi灬",
	223342537: "lupsione",
	117514269: "laise",
	399804216: "Serenada",
	238239590: "Bryle",
	439345730: "reibl",
	392006194: "ififall",
	333351042: "mrls",
	9403474: "yamich",
	910028605: "Unknown",
	114619230: "Unknown",
	296702734: "Unknown",
	276449590: "Unknown",
	1048617659: "Unknown",
	200573858: "Unknown"
}

# Update player names in df_players where account_id matches
df_players.loc[df_players['account_id'].isin(account_id_to_name.keys()), 'name'] = \
	df_players.loc[df_players['account_id'].isin(account_id_to_name.keys()), 'account_id'].map(account_id_to_name)

In [119]:
# df_players[df_players['account_id'].isin(df_players_name_none_unique.tolist())][['account_id', 'match_id', 'player_slot', 'gold_per_min', 'xp_per_min']].drop_duplicates('account_id')

To do:
- Fix missing player info like in exploration.ipynb afer full dataset is available.

## Enrich playerdata with fantasy role

In [120]:
with open('data/proPlayers.json', 'r', encoding='utf-8') as f:
	pro_players_data = json.load(f)

df_fantasy_role = pd.DataFrame(pro_players_data)[['account_id', 'fantasy_role']]


In [121]:
df_players = df_players.merge(df_fantasy_role, on='account_id', how='left')


In [122]:
player_data_filter = [
    # Metadata
	'match_id',
    'match_start_time',
	'account_id',
    'name',
	'fantasy_role',
    'hero_id',
    'hero_variant',
    'isRadiant',
    'radiant_team_id',
	'dire_team_id',
	'tournament_start_date',
    'duration',
    'win',
    # FP Stats
	'kills',
	'deaths',
    'last_hits',
	'denies',
    'gold_per_min',
	'towers_killed',
	'roshans_killed',
	'teamfight_participation',
    'obs_placed',
	'camps_stacked',
	'rune_pickups',
	'firstblood_claimed',
	'stuns',
    # Additional Stats
	'creeps_stacked',
    'assists',
	'xp_per_min',
    'lane_efficiency',
]

In [123]:
df_players = df_players[player_data_filter]

In [124]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15530 entries, 0 to 15529
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  int64  
 1   match_start_time         15530 non-null  int64  
 2   match_start_time         15530 non-null  int64  
 3   account_id               15530 non-null  int64  
 4   name                     15530 non-null  object 
 5   fantasy_role             14666 non-null  float64
 6   hero_id                  15530 non-null  int64  
 7   hero_variant             15530 non-null  int64  
 8   isRadiant                15530 non-null  bool   
 9   radiant_team_id          15530 non-null  int64  
 10  dire_team_id             15530 non-null  int64  
 11  tournament_start_date    15530 non-null  object 
 12  duration                 15530 non-null  int64  
 13  win                      15530 non-null  int64  
 14  kills                 

In [125]:
df_players.head(2)

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency
0,7944311818,1726415990,1726415990,91730177,Seleri,2.0,136,1,True,8599101,2163,2024-09-04,2068,0,3,7,36,2,248,0,0,0.647059,7,2,3,0,29.466703,6,8,379,0.418957
1,7944311818,1726415990,1726415990,116934015,dyrachyo,1.0,21,3,True,8599101,2163,2024-09-04,2068,0,4,7,275,7,540,0,0,0.705882,0,7,0,0,71.866130,22,8,527,0.618634


In [126]:
# Enrich player data with team_id based on isRadiant flag
df_players['team_id'] = df_players.apply(lambda row: row['radiant_team_id'] if row['isRadiant'] else row['dire_team_id'], axis=1)
df_players['enemy_team_id'] = df_players.apply(lambda row: row['dire_team_id'] if row['isRadiant'] else row['radiant_team_id'], axis=1)



In [127]:
def assign_glicko_rating(df: pd.DataFrame, date: str):
	with open(f'data/ratings/ratings_{date}.json', 'r', encoding='utf-8') as f:
		glicko_ratings = pd.DataFrame(json.load(f))
	df = df[df['tournament_start_date'] == date].copy()
	df['team_id'] = df['team_id'].astype('Int64').astype(str)
	df['enemy_team_id'] = df['enemy_team_id'].astype('Int64').astype(str)
	glicko_ratings['valveId'] = glicko_ratings['valveId'].astype('Int64').astype(str)
	df = df.merge(glicko_ratings, left_on='team_id', right_on='valveId', how='left', suffixes=('', '_team'))
	df = df.merge(glicko_ratings, left_on='enemy_team_id', right_on='valveId', how='left', suffixes=('', '_enemy'))
	# Rename the columns from the second merge with prefix 'enemy_'
	for col in ['teamName', 'valveId', 'glicko2_rating']:
		df.rename(columns={f'{col}_enemy': f'enemy_{col}'}, inplace=True)
	return df

In [128]:
# Collect all unique dates
unique_dates = df_players['tournament_start_date'].unique()

# List to store the resulting dataframes
df_glicko_list = []

for date in unique_dates:
	df_glicko = assign_glicko_rating(df_players, date)
	df_glicko_list.append(df_glicko)

# Concatenate all dataframes into one
df_players_with_glicko = pd.concat(df_glicko_list, ignore_index=True)

In [129]:
df_players_with_glicko.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15530 entries, 0 to 15529
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  int64  
 1   match_start_time         15530 non-null  int64  
 2   match_start_time         15530 non-null  int64  
 3   account_id               15530 non-null  int64  
 4   name                     15530 non-null  object 
 5   fantasy_role             14666 non-null  float64
 6   hero_id                  15530 non-null  int64  
 7   hero_variant             15530 non-null  int64  
 8   isRadiant                15530 non-null  bool   
 9   radiant_team_id          15530 non-null  int64  
 10  dire_team_id             15530 non-null  int64  
 11  tournament_start_date    15530 non-null  object 
 12  duration                 15530 non-null  int64  
 13  win                      15530 non-null  int64  
 14  kills                 

Drop data that can not be assigned a Glicko Score.

In [130]:
#df_players_with_glicko.dropna(axis=0, inplace=True)

In [131]:
df_players_with_glicko[df_players_with_glicko['valveId'].isna()]

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating
1440,7982874526,1728650424,1728650424,96803083,Stormstormer,4.0,74,2,True,9498970,8254400,2024-10-04,1945,0,2,6,170,19,381,0,0,0.888889,2,0,4,0,32.135303,0,6,399,0.791835,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1441,7982874526,1728650424,1728650424,369342470,SoNNeikO,NaN,5,1,True,9498970,8254400,2024-10-04,1945,0,1,8,30,1,232,0,0,0.444444,11,1,1,0,57.199802,3,3,247,0.265764,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1442,7982874526,1728650424,1728650424,155162307,Smiling Knight,NaN,70,2,True,9498970,8254400,2024-10-04,1945,0,1,4,290,4,527,0,0,0.222222,0,1,5,0,0.000000,3,1,556,0.741512,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1443,7982874526,1728650424,1728650424,230487729,Ekki,2.0,73,1,True,9498970,8254400,2024-10-04,1945,0,1,7,107,4,313,0,0,0.555556,3,4,2,0,50.566140,10,4,305,0.416330,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1444,7982874526,1728650424,1728650424,50580004,Xibbe,1.0,92,3,True,9498970,8254400,2024-10-04,1945,0,4,8,128,9,354,1,0,0.777778,0,2,2,1,24.031889,8,3,318,0.612369,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15520,8366140862,1751966193,1751966193,957204049,gotthejuice,1.0,102,1,True,9828954,8597976,2025-07-08,3080,1,13,3,669,16,901,1,1,0.838710,0,2,3,0,35.401466,3,13,1134,0.785974,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231
15521,8366140862,1751966193,1751966193,185590374,Niku,1.0,11,2,True,9828954,8597976,2025-07-08,3080,1,12,6,563,9,781,1,0,0.806452,1,2,11,1,34.614555,5,13,1103,0.852061,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231
15522,8366140862,1751966193,1751966193,835864135,pma,1.0,120,1,True,9828954,8597976,2025-07-08,3080,1,5,3,442,4,639,0,1,0.935484,0,4,1,0,8.685791,7,24,1038,0.604285,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231
15523,8366140862,1751966193,1751966193,111030315,KG_Zayac,2.0,27,2,True,9828954,8597976,2025-07-08,3080,1,0,10,72,0,314,4,0,0.580645,9,8,3,0,124.567920,35,18,460,0.268998,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231


In [132]:
df_players_with_glicko.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15530 entries, 0 to 15529
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  int64  
 1   match_start_time         15530 non-null  int64  
 2   match_start_time         15530 non-null  int64  
 3   account_id               15530 non-null  int64  
 4   name                     15530 non-null  object 
 5   fantasy_role             14666 non-null  float64
 6   hero_id                  15530 non-null  int64  
 7   hero_variant             15530 non-null  int64  
 8   isRadiant                15530 non-null  bool   
 9   radiant_team_id          15530 non-null  int64  
 10  dire_team_id             15530 non-null  int64  
 11  tournament_start_date    15530 non-null  object 
 12  duration                 15530 non-null  int64  
 13  win                      15530 non-null  int64  
 14  kills                 

In [133]:
df_players_with_glicko.head(11)

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating
0,7944311818,1726415990,1726415990,91730177,Seleri,2.0,136,1,True,8599101,2163,2024-09-04,2068,0,3,7,36,2,248,0,0,0.647059,7,2,3,0,29.466703,6,8,379,0.418957,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
1,7944311818,1726415990,1726415990,116934015,dyrachyo,1.0,21,3,True,8599101,2163,2024-09-04,2068,0,4,7,275,7,540,0,0,0.705882,0,7,0,0,71.866130,22,8,527,0.618634,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
2,7944311818,1726415990,1726415990,97590558,Ace ♠,1.0,108,2,True,8599101,2163,2024-09-04,2068,0,2,5,208,5,408,1,0,0.647059,0,2,1,0,104.965775,4,9,499,0.942603,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
3,7944311818,1726415990,1726415990,16497807,tOfu,2.0,9,2,True,8599101,2163,2024-09-04,2068,0,4,8,125,0,312,0,0,0.647059,7,5,6,0,31.299086,10,7,378,0.529709,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
4,7944311818,1726415990,1726415990,221666230,Quinn,4.0,120,1,True,8599101,2163,2024-09-04,2068,0,4,7,207,7,443,0,0,0.705882,1,0,5,0,0.319238,0,8,614,0.840137,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
5,7944311818,1726415990,1726415990,54580962,Insania,2.0,51,2,False,8599101,2163,2024-09-04,2068,1,1,4,47,2,334,0,0,0.764706,7,4,9,0,45.298760,12,25,454,0.336904,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
6,7944311818,1726415990,1726415990,86698277,33,1.0,92,3,False,8599101,2163,2024-09-04,2068,1,6,3,127,11,571,4,0,0.676471,0,4,4,1,40.065290,14,17,610,0.799313,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
7,7944311818,1726415990,1726415990,201358612,Nisha,4.0,13,1,False,8599101,2163,2024-09-04,2068,1,11,1,299,8,679,1,0,0.852941,2,2,18,0,15.833374,4,18,843,0.854487,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
8,7944311818,1726415990,1726415990,77490514,Boxi,2.0,119,1,False,8599101,2163,2024-09-04,2068,1,5,4,29,2,356,0,0,0.647059,7,8,4,0,146.532930,44,18,461,0.296079,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
9,7944311818,1726415990,1726415990,152962063,m1CKe,1.0,53,2,False,8599101,2163,2024-09-04,2068,1,11,5,344,9,810,4,1,0.941176,0,4,1,0,58.966675,10,21,868,0.755053,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489


In [134]:
df_players = df_players_with_glicko.copy()

# Fantasy Calculation

In [135]:
def calculate_fantasy_score(row):
	return (
		row['kills'] * 0.3 +
		(3 - row['deaths'] * 0.3) +
		(row['last_hits'] + row['denies']) * 0.003 +
		row['gold_per_min'] * 0.002 +
		row['towers_killed'] +
		row['roshans_killed'] +
		row['teamfight_participation'] * 3 +
		row['obs_placed'] * 0.5 +
		row['camps_stacked'] * 0.5 +
		row['rune_pickups'] * 0.25 +
		row['firstblood_claimed'] * 4 +
		row['stuns'] * 0.05
	)

df_players['fantasy_score'] = df_players.apply(calculate_fantasy_score, axis=1)

In [136]:
df_players.sample(11)

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score
6672,8082223614,1734238850,1734238850,201358612,Nisha,4.0,13,1,True,2163,8255888,2024-12-09,2190,0,6,8,272,9,587,1,0,0.583333,2,0,17,0,3.500000,0,8,749,0.926233,2163,8255888,Team Liquid,2163,2014.669359,BetBoom Team,8255888,2040.857811,12.592000
8967,8178209050,1739807940,1739807940,165564598,MieRo,1.0,19,1,False,7422789,8255888,2025-02-16,1917,1,9,2,217,5,625,2,0,0.605263,1,1,3,0,59.627697,3,14,738,0.731407,8255888,7422789,BetBoom Team,8255888,2022.786385,9Pandas,7422789,1731.653566,15.563174
5730,8054644399,1732615249,1732615249,1044002267,Satanic,1.0,72,2,True,7119388,8599101,2024-11-26,2177,0,0,4,361,11,552,1,0,0.714286,0,6,3,0,6.466406,15,5,585,0.745352,7119388,8599101,Team Spirit,7119388,1973.878587,Gaimin Gladiators,8599101,1953.690463,11.236178
15490,8366209805,1751971261,1751971261,957204049,gotthejuice,1.0,18,2,True,9828954,8597976,2025-07-08,2631,0,8,3,593,7,767,0,0,0.619048,0,2,3,0,28.099821,4,5,953,0.927648,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231,12.846134
48,7943704335,1726392921,1726392921,16497807,tOfu,2.0,9,2,False,8291895,8599101,2024-09-04,1818,1,3,3,72,1,356,0,0,0.793103,6,4,3,0,42.132810,16,20,428,0.392684,8599101,8291895,Gaimin Gladiators,8599101,1980.466489,Tundra Esports,8291895,2002.501023,14.166951
6531,8282581557,1746531222,1746531222,858106446,kiyotaka,4.0,17,2,True,9467224,7119388,2025-05-06,2304,0,5,9,245,7,436,0,0,0.800000,1,0,11,0,19.366650,0,8,511,0.604487,9467224,7119388,Aurora Gaming,9467224,1918.366675,Team Spirit,7119388,2061.664793,10.046333
9808,8248399016,1744224910,1744224910,155332459,Matthew,2.0,138,1,False,8291895,9303484,2025-04-07,1953,0,2,4,44,6,219,0,0,0.750000,1,1,1,1,40.399742,2,4,233,0.393897,9303484,8291895,HEROIC,9303484,1845.246483,Tundra Esports,8291895,2072.225757,12.507987
4054,8015814406,1730557381,1730557381,103735745,Saksa,2.0,51,2,True,8291895,7119388,2024-10-27,2024,0,8,11,45,4,279,0,0,0.800000,6,4,8,0,57.799040,15,8,365,0.699070,8291895,7119388,Tundra Esports,8291895,2013.159488,Team Spirit,7119388,1965.054843,15.094952
6501,8282687386,1746537015,1746537015,173978074,NothingToSay,4.0,120,2,True,9640842,8261500,2025-05-06,2337,1,10,0,284,13,606,0,0,0.962963,1,0,19,0,9.469627,0,16,780,0.956952,9640842,8261500,Team Tidebound,9640842,2004.109120,Xtreme Gaming,8261500,1902.002415,16.715370
12960,8311170339,1748424345,1748424345,168028715,flyfly,1.0,63,1,True,9351740,9467224,2025-05-19,2340,0,6,3,381,17,628,2,0,0.666667,0,0,0,0,0.000000,0,4,762,0.773646,9351740,9467224,Yakult Brothers,9351740,1894.054526,Aurora Gaming,9467224,1942.349787,10.350000


In [137]:
# Create feature showing the rating advantage of the player
df_players['rating_advantage'] = df_players['glicko2_rating'] - df_players['enemy_glicko2_rating']

In [138]:
# Create feature to make hero_variant useable
# Das muss gemacht werden, damit der Kontext von hero + facet korrekt ist.
df_players['hero_variant'] = df_players['hero_id'].astype(str) + ", " + df_players['hero_variant'].astype(str)

In [139]:
df_players['fantasy_role'].value_counts()

fantasy_role
1.0    5830
2.0    5741
4.0    3057
0.0      38
Name: count, dtype: int64

In [140]:
# df_players[df_players['fantasy_role'] == 0]

In [141]:
# There are two players with an invalid fantasy role (0). Manually set them to the correct role.
df_players.loc[df_players['account_id'] == 1657135701, 'fantasy_role'] = 1
df_players.loc[df_players['account_id'] == 489696354, 'fantasy_role'] = 4

## Make roles human readable

In [142]:
df_players['fantasy_role'] = df_players['fantasy_role'].astype(str)
df_players['fantasy_role'] = df_players['fantasy_role'].replace({'1.0': 'Core', '2.0': 'Support', '4.0': 'Mid'})

In [143]:
df_players['fantasy_role']

0        Support
1           Core
2           Core
3        Support
4            Mid
          ...   
15525    Support
15526        Mid
15527    Support
15528       Core
15529       Core
Name: fantasy_role, Length: 15530, dtype: object

In [144]:
df_players_test = df_players.astype({
    'match_id': 'string',
    'account_id': 'string',
    'hero_id': 'string',
    'radiant_team_id': 'string',
    'dire_team_id': 'string',
    'team_id': 'string',
	'enemy_team_id': 'string',
    'valveId': 'string',
})

In [145]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15530 entries, 0 to 15529
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15530 non-null  int64  
 1   match_start_time         15530 non-null  int64  
 2   match_start_time         15530 non-null  int64  
 3   account_id               15530 non-null  int64  
 4   name                     15530 non-null  object 
 5   fantasy_role             15530 non-null  object 
 6   hero_id                  15530 non-null  int64  
 7   hero_variant             15530 non-null  object 
 8   isRadiant                15530 non-null  bool   
 9   radiant_team_id          15530 non-null  int64  
 10  dire_team_id             15530 non-null  int64  
 11  tournament_start_date    15530 non-null  object 
 12  duration                 15530 non-null  int64  
 13  win                      15530 non-null  int64  
 14  kills                 

In [153]:
df_players = df_players.loc[:, ~df_players.columns.duplicated()] 

In [154]:
df_players[~df_players['rating_advantage'].isna()].info()
df_players_clean = df_players[~df_players['rating_advantage'].isna()]

<class 'pandas.core.frame.DataFrame'>
Index: 12760 entries, 0 to 15519
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 12760 non-null  int64  
 1   match_start_time         12760 non-null  int64  
 2   account_id               12760 non-null  int64  
 3   name                     12760 non-null  object 
 4   fantasy_role             12760 non-null  object 
 5   hero_id                  12760 non-null  int64  
 6   hero_variant             12760 non-null  object 
 7   isRadiant                12760 non-null  bool   
 8   radiant_team_id          12760 non-null  int64  
 9   dire_team_id             12760 non-null  int64  
 10  tournament_start_date    12760 non-null  object 
 11  duration                 12760 non-null  int64  
 12  win                      12760 non-null  int64  
 13  kills                    12760 non-null  int64  
 14  deaths                   12

In [155]:
df_players.head()

,match_id,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score,rating_advantage
0,7944311818,1726415990,91730177,Seleri,Support,136,"136, 1",True,8599101,2163,2024-09-04,2068,0,3,7,36,2,248,0,0,0.647059,7,2,3,0,29.466703,6,8,379,0.418957,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.60708,11.074512,20.85941
1,7944311818,1726415990,116934015,dyrachyo,Core,21,"21, 3",True,8599101,2163,2024-09-04,2068,0,4,7,275,7,540,0,0,0.705882,0,7,0,0,71.866130,22,8,527,0.618634,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.60708,13.236954,20.85941
2,7944311818,1726415990,97590558,Ace ♠,Core,108,"108, 2",True,8599101,2163,2024-09-04,2068,0,2,5,208,5,408,1,0,0.647059,0,2,1,0,104.965775,4,9,499,0.942603,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.60708,12.994465,20.85941
3,7944311818,1726415990,16497807,tOfu,Support,9,"9, 2",True,8599101,2163,2024-09-04,2068,0,4,8,125,0,312,0,0,0.647059,7,5,6,0,31.299086,10,7,378,0.529709,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.60708,13.805131,20.85941
4,7944311818,1726415990,221666230,Quinn,Mid,120,"120, 1",True,8599101,2163,2024-09-04,2068,0,4,7,207,7,443,0,0,0.705882,1,0,5,0,0.319238,0,8,614,0.840137,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.60708,7.511609,20.85941


# Write data to json files

In [156]:
df_players.to_json('data/curated/player_data.json', index=False)

In [157]:
df_players_clean.to_json('data/curated/player_data_clean.json', index=False)